In the following, we present a case to show how to incorporate the personalized loss and plug it into different DRO losses.

In [3]:
import os
# Set the new path
new_path = "/Users/wty6162/Desktop/DRO_solver/dro"

# Change the current working directory
os.chdir(new_path)

import numpy as np

#from dro.src.OR_Wasserstein_DRO import *
from dro.src.Wasserstein_DRO import *
from dro.src.chi2_dro import *

X = np.array([[1, 1], [2, 1], [3, 1], [4,1]])
y = np.array([1, 1, 0, 0])
model = Chi2DRO(input_dim = 2, model_type = 'quantile')


from types import MethodType

def _loss(self, X, y):
    return 3 * np.maximum(y - X @ self.theta, 0) + np.maximum(X @ self.theta - y, 0)

def _cvx_loss(self, X, y, theta):
    return 3 * cp.pos(y - X @ theta) + 1 * cp.pos(X @ theta - y)

model._loss = MethodType(_loss, model)
model._cvx_loss = MethodType(_cvx_loss, model)
for k in range(5):
    model.update({'eps': 0.05 * (k + 1), 'n_certifiy_ratio': 1})
    print(model.fit(X, y))

{'theta': [-0.48891007299787476, 1.9556403007778032]}
{'theta': [-0.47103180655319055, 1.8841272540151666]}
{'theta': [-0.4651670885530432, 1.8606683558961619]}
{'theta': [-0.4620192279713162, 1.8480769103478247]}
{'theta': [-0.4599744797040242, 1.839897918619857]}


However, for more complicated losses, such as hr-dro and wasserstein-dro, where the loss difference changes with the inner function fit, it has not been implemented yet.

In [5]:
np.log(1 + np.exp(-np.multiply(y, X @ model.theta)))

array([0.22442099, 0.33542839, 0.69314718, 0.69314718])

Uncontextual and Contextual Robust Learning:
- uncontextual: by taking $X$ as a 1-d unit one vector
- contextual: newsvendor (the same as quantile regression); portfolio 